In [1]:
import torch

data_path = "../data/reduced_train_test_split//"

X_train_tensor = torch.load(data_path + "X_train.pt")
y_train_tensor = torch.load(data_path + "y_train.pt")
X_test_tensor = torch.load(data_path + "X_test.pt")
y_test_tensor = torch.load(data_path + "y_test.pt")

print(X_train_tensor[:5])
print(y_train_tensor[:5])
print(X_test_tensor[:5])
print(y_test_tensor[:5])

C:\Users\gurgel\AppData\Local\Temp\ipykernel_23400\3177810588.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train_tensor = torch.load(data_path + "X_train.pt")
C:\Us

tensor([[ 4.4000e+01,  3.8642e-01,  2.9976e-01,  2.1417e-01,  1.3460e-01,
          4.1206e-02, -5.1461e-02, -5.5842e-02, -1.5758e-01, -1.7905e-01,
         -2.1703e-01, -2.0291e-01, -1.9350e-01, -1.5718e-01, -3.3163e-02,
          1.0322e-01,  2.0906e-01,  3.6304e-01,  5.2003e-01,  7.1807e-01,
          1.1197e+00,  1.3429e+00,  1.5217e+00,  1.7501e+00,  1.9403e+00,
          1.8880e+00,  3.4217e+00,  2.1180e+00,  2.1980e+00,  2.1193e+00,
          1.9848e+00,  1.8394e+00,  1.8786e+00,  1.7225e+00,  1.4780e+00,
          1.4555e+00,  1.1440e+00,  9.0688e-01,  7.6223e-01,  4.3167e-01,
          2.0761e-01,  1.8334e-02, -8.5021e-02, -2.3845e-01, -4.3698e-01,
         -5.8092e-01, -7.2828e-01, -1.0801e+00, -1.2079e+00, -1.2878e+00,
         -1.4551e+00, -1.5468e+00, -5.2357e-01, -5.6683e-01],
        [ 2.6000e+01, -6.9760e-01, -6.8235e-01, -6.4446e-01, -6.4049e-01,
         -6.1060e-01, -5.8075e-01, -5.4399e-01, -5.4720e-01, -5.2102e-01,
         -4.8339e-01, -4.4630e-01, -4.3467e-01, -4

C:\Users\gurgel\AppData\Local\Temp\ipykernel_23400\3177810588.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  y_test_tensor = torch.load(data_path + "y_test.pt")


In [2]:
from torch.utils.data import DataLoader, TensorDataset

inputs = X_train_tensor.shape[1]
batch_size = 512
num_workers = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

print(train_loader.dataset.tensors)
print(test_loader.dataset.tensors)

(tensor([[ 4.4000e+01,  3.8642e-01,  2.9976e-01,  ..., -1.5468e+00,
         -5.2357e-01, -5.6683e-01],
        [ 2.6000e+01, -6.9760e-01, -6.8235e-01,  ...,  3.9905e-01,
          8.6546e-01,  8.5090e-01],
        [ 5.2000e+01, -6.3395e-01, -6.4926e-01,  ...,  4.0282e-01,
         -1.1731e+00, -1.1522e+00],
        ...,
        [ 4.0000e+00,  1.2202e+00,  1.2272e+00,  ...,  1.4772e+00,
          1.6093e+00,  1.5470e+00],
        [ 1.9000e+01, -1.7210e+00, -1.6329e+00,  ...,  1.5486e+00,
          1.6795e+00,  1.7027e+00],
        [ 4.0000e+00, -3.7073e-01, -3.5631e-01,  ..., -1.9827e-02,
          5.4358e-01,  5.1449e-01]]), tensor([17, 12,  8,  ...,  2,  5, 12]))
(tensor([[46.0000, -0.2138, -0.2730,  ..., -0.5477, -1.1744, -1.1432],
        [10.0000, -1.2046, -1.1377,  ...,  0.5050, -1.1571, -1.1429],
        [33.0000, -1.7444, -1.7738,  ...,  0.8641,  1.2261,  1.2260],
        ...,
        [35.0000,  1.3852,  1.5208,  ...,  1.7421, -1.1643, -1.1480],
        [56.0000, -0.8822, -0.91

In [3]:
from mlp import CsiNeuralNet

checkpoint_model = "mlp/reduced_model_epoch_437.pth"
save_path = "mlp/reduced_model.pth"
num_epochs = 1000
net = CsiNeuralNet(inputs, device, state_file=None)

result_model = net.train_model(train_loader, test_loader, save_path, num_epochs)

Training on device: cuda
Epoch 1/1000, Accuracy: 34.40%, Loss: 2.13760736, In 30.28s
Model saved for epoch 9 with accuracy 74.96%
Epoch 10/1000, Accuracy: 75.35%, Loss: 0.82307343, In 298.26s
Model saved for epoch 20 with accuracy 80.59%
Epoch 20/1000, Accuracy: 80.69%, Loss: 0.62951007, In 336.99s
Model saved for epoch 28 with accuracy 82.18%
Epoch 30/1000, Accuracy: 82.73%, Loss: 0.55330164, In 358.73s
Model saved for epoch 40 with accuracy 83.18%
Epoch 40/1000, Accuracy: 83.98%, Loss: 0.50647937, In 333.31s
Model saved for epoch 46 with accuracy 83.81%
Epoch 50/1000, Accuracy: 84.78%, Loss: 0.47549092, In 333.95s
Model saved for epoch 60 with accuracy 84.42%
Epoch 60/1000, Accuracy: 85.48%, Loss: 0.44967554, In 334.23s
Model saved for epoch 70 with accuracy 84.84%
Epoch 70/1000, Accuracy: 86.04%, Loss: 0.42929100, In 314.19s
Model saved for epoch 80 with accuracy 85.43%
Epoch 80/1000, Accuracy: 86.52%, Loss: 0.41122270, In 300.51s
Model saved for epoch 82 with accuracy 85.78%
Epoch 